In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 8-3. Expr型とシンボル

### 8-3-1. AST

### 8-3-2. Expr型

#### コード8-1. `:(～)` の例(1)

In [2]:
1 + 2

3

In [3]:
a + b

LoadError: UndefVarError: a not defined

In [4]:
:(1 + 2)

:(1 + 2)

In [5]:
:(a + b)

:(a + b)

In [6]:
:(x^2+2x-1)

:((x ^ 2 + 2x) - 1)

#### コード8-2. `:(～)` の例(2)：`:(a + b)` の調査

In [7]:
ex1 = :(a + b)

:(a + b)

In [8]:
typeof(ex1)

Expr

In [9]:
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b


In [10]:
ex1.head

:call

In [11]:
ex1.args

3-element Vector{Any}:
 :+
 :a
 :b

#### コード8-3. `:(～)` の例(3)：`:(a * b + c / d)` の調査

In [12]:
ex2 = :(a * b + c / d)

:(a * b + c / d)

In [13]:
dump(ex2)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Symbol a
        3: Symbol b
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol /
        2: Symbol c
        3: Symbol d


#### コード8-4. 複数行の式からなる `Expr`（`quote ～ end`）の例

In [14]:
:(
    a = 1
    b = 2
    println(a + b)
)

LoadError: syntax: missing comma or ) in argument list

In [15]:
quote
    a = 1
    b = 2
    println(a + b)
end

quote
    #= In[15]:2 =#
    a = 1
    #= In[15]:3 =#
    b = 2
    #= In[15]:4 =#
    println(a + b)
end

In [16]:
:(begin
    a = 1
    b = 2
    println(a + b)
end)  # 上記と全く同等

quote
    #= In[16]:2 =#
    a = 1
    #= In[16]:3 =#
    b = 2
    #= In[16]:4 =#
    println(a + b)
end

In [17]:
ex4 = quote
    a = 1
    b = 2
    println(a + b)
end;

Base.remove_linenums!(ex4)  # LinenumberNode を除去

quote
    a = 1
    b = 2
    println(a + b)
end

In [18]:
typeof(ex4)

Expr

In [19]:
ex4.head

:block

In [20]:
ex4.args

3-element Vector{Any}:
 :(a = 1)
 :(b = 2)
 :(println(a + b))

#### コード8-5. `Expr` コンストラクタ例

In [21]:
Expr(:if, :(a > b), :(println(a)), :(println(b)))

:(if a > b
      println(a)
  else
      println(b)
  end)

In [22]:
Base.remove_linenums!(:(if a > b println(a) else println(b) end))  # 参考

:(if a > b
      println(a)
  else
      println(b)
  end)

### 8-3-3. シンボル

#### コード8-6. `:(～)` の例(4)：単一の値や識別子を渡した場合の挙動

In [23]:
:(1)

1

In [24]:
typeof(:(1))

Int64

In [25]:
:("123ABCあいう漢字😁")

"123ABCあいう漢字😁"

In [26]:
typeof(:("123ABCあいう漢字😁"))

String

In [27]:
:(a)

:a

In [28]:
typeof(:(a))

Symbol

### 8-3-4. `QuoteNode`

#### コード8-7. QuoteNode の例(1)

In [29]:
:(:a)

:(:a)

In [30]:
typeof(:(:a))

QuoteNode

In [31]:
dump(:(:a))

QuoteNode
  value: Symbol a


#### コード8-8. QuoteNode の例(2)

In [32]:
dump(:(d[:key]))  # 辞書（`Dict`）のキーとしてシンボルを指定した例

Expr
  head: Symbol ref
  args: Array{Any}((2,))
    1: Symbol d
    2: QuoteNode
      value: Symbol key


In [33]:
dump(:(a.b))  # プロパティ参照（`b`は識別子ではなくシンボルとして扱われる）

Expr
  head: Symbol .
  args: Array{Any}((2,))
    1: Symbol a
    2: QuoteNode
      value: Symbol b
